# Data Visualization Dashboard Project

## Import packages

In [1]:
import eurostat
import pandas as pd
import openpyxl


In [2]:
import plotly.graph_objects as go
import numpy as np

In [3]:
data_out = '../data/'

## Codes

In [4]:
siec_codes = eurostat.get_dic('siec')

geo_eu = [
'BE','BG','CZ','DK','DE',
'EE','IE','EL','ES','FR',
'HR','IT','CY','LV','LT',
'LU','HU','MT','NL','AT',
'PL','PT','RO','SI','SK',
'FI','SE','IS','LI','NO'
'CH',
]

geo_codes = {
    'BE': 'Belgium',
    'BG': 'Bulgaria',
    'CZ': 'Czechia',
    'DK': 'Denmark',
    'DE': 'Germany',
    'EE': 'Estonia',
    'IE': 'Ireland',
    'EL': 'Greece',
    'ES': 'Spain',
    'FR': 'France',
    'HR': 'Croatia',
    'IT': 'Italy',
    'CY': 'Cyprus',
    'LV': 'Latvia',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'HU': 'Hungary',
    'MT': 'Malta',
    'NL': 'Netherlands',
    'AT': 'Austria',
    'PL': 'Poland',
    'PT': 'Portugal',
    'RO': 'Romania',
    'SI': 'Slovenia',
    'SK': 'Slovakia',
    'FI': 'Finland',
    'SE': 'Sweden',
    'IS': 'Iceland',
    'LI': 'Liechtenstein',
    'NO': 'Norway',
    'CH': 'Switzerland',
    'UK': 'United Kingdom'
}

x = ['R5110-5150_W6000RI',
 'R5200',
 'R5300',
 'RA000',
 'RA200',
 'RA410',
 'RA500_5160',
 'RA600',
 'W6210'
]

rs_codes = [
 'R5110-5150_W6000RI',
 'R5200', 
 'R5300', 
 'RA100',
 'RA200',
 'RA300',
 'RA410',
 'RA420', 
 'RA500_5160', 
 'RA600',
 'W6210',
# 'RA000',
           ]



## Get SIEC Timeseries

In [5]:
es_code = 'NRG_IND_COTD'
data = eurostat.get_data_df(str.lower(es_code), flags=False)

In [6]:
## Rename columns:
data.rename(columns={'geo\\time': 'Country Code', 'siec':'SIEC Code'}, inplace=True)

In [7]:
# Keep only selected countries
data = data.loc[data['Country Code'].isin(geo_eu),:]

In [8]:
# Remove extra rows in nrg_bal
# Keep only:
# Gross final consumption - energy use - overall target calculation without aviation cap adjustment 
# [GFC_E_OT_NAVICAPA]

data = data.loc[data['nrg_bal']=='GFC_E_OT_NAVICAPA',:]


In [9]:
# Remove unneccessary cols 
data.drop(columns=['nrg_bal','unit'], inplace=True)

In [10]:
data.reset_index(drop=True, inplace=True)

In [11]:
data['Country'] = data['Country Code']
data['Country'] = data['Country'].map(geo_codes)

In [12]:
data['SIEC'] = data['SIEC Code']
data['SIEC'] = data['SIEC'].map(siec_codes)

In [13]:
data.to_csv(data_out + 'renewables_percent_timeseries.csv', index=False)

## Get Energy Flow Data 

In [14]:
sk_code = 'NRG_BAL_SD'
skdata = eurostat.get_data_df(str.lower(sk_code), flags=False)

In [15]:
## AAS = Available from all sources
renewable_sources2020 = skdata.loc[skdata['geo\\time'].isin(geo_eu)].loc[skdata['nrg_bal']=='AAS',['geo\\time', 'nrg_bal', 'siec', 2020]]
renewable_sources2020.rename(columns={'geo\\time': 'Country Code'}, inplace=True)
renewable_sources2020.dropna(inplace=True)


renewable_sources2020 = renewable_sources2020.loc[renewable_sources2020['siec'].isin(rs_codes)]

renewable_sources2020['SIEC'] = renewable_sources2020['siec'].map(siec_codes)

renewable_sources2020 = renewable_sources2020.rename(columns={
                                         2020: 'Consumption in KTOE',
                                         'siec': 'SIEC Code'
                                        }).reset_index(drop=True).drop(columns=['nrg_bal'])


renewable_sources2020.to_csv(data_out + 'renewable_sources_2020.csv', index=False)


In [16]:
# renewable_sources2020.loc[renewable_sources2020['Country Code']=='AT','Consumption in KTOE'].sum()
# renewable_sources2020.loc[renewable_sources2020['Country Code']=='AT']

In [17]:
renewable_sources2020['Country Code'].unique()

array(['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI',
       'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL',
       'PT', 'RO', 'SE', 'SI', 'SK'], dtype=object)